# Data Screening

In [1]:
import pandas as pd
import numpy as np
import os
import time

In [29]:
df_raw = pd.read_csv("data/BLACKKNIGHT_ANALYSIS_SUBSET.csv", index_col = [0], low_memory = False)

#np.sort(df_raw.standardized_land_use_code.unique())

#dict(df_raw.standardized_land_use_code.value_counts())

## Removing records with land_use_code of 8000 level, i.e. vacant land

In [30]:
df = df_raw
df = df[ (df['standardized_land_use_code'] < 8000) | (df['standardized_land_use_code'] >= 9000) | df['standardized_land_use_code'].isna() ]

## dpid is the primary key.

In [31]:
# pop the primary key to the first column
first_col = df.pop('dpid')

df.insert(0, 'dpid', first_col)

#df = df.iloc[ : , : 22]

#df.head()

### Generate a column full_address = street + city + state + zipcode
The full address can help us differentiate records with the same street addresses but in different cities or states.

There are some records with street addresses and zipcode but missing state and city. So we use the zipcode to fill in the null state and city columns.

In [32]:
zipcode = pd.read_csv("data/zip_code_database.csv", dtype = {'zip': str})

In [33]:
zipcode.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
0,00501,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,00544,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,00601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787, 939",NaN,US,18.16,-66.72,0
3,00602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787, 939",NaN,US,18.38,-67.18,0
4,00603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,"787, 939",NaN,US,18.43,-67.15,0


In [34]:
df['property_zip_code'] = df['property_zip_code'].fillna(0).astype(int).astype(str).replace('0', np.nan)

<ipython-input-34-3cdb874e2525>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['property_zip_code'] = df['property_zip_code'].fillna(0).astype(int).astype(str).replace('0', np.nan)


In [35]:
zip_state_dict = dict(list(zip(zipcode.zip, zipcode.state)))
zip_city_dict = dict(list(zip(zipcode.zip, zipcode.primary_city.str.upper())))
#zip_city_dict

In [36]:
df.property_city_name.isna().sum()

93021

In [37]:
#df.property_state.unique()

In [38]:
df['property_city_name'] = df['property_city_name'].fillna(df['property_zip_code'].map(zip_city_dict))
df['property_state'] = df['property_state'].fillna(df['property_zip_code'].map(zip_state_dict))
#fillna(df['country'].map(country_gdp_dict))

<ipython-input-38-cc43afd9050e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['property_city_name'] = df['property_city_name'].fillna(df['property_zip_code'].map(zip_city_dict))
<ipython-input-38-cc43afd9050e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['property_state'] = df['property_state'].fillna(df['property_zip_code'].map(zip_state_dict))


In [39]:
df.property_city_name.isna().sum()

16903

In [40]:
df.property_state.isna().sum()

16837

There are records with no city, no state and no zipcode. For these records, we extract the state info from the fips code.

In [41]:
fips_state = pd.read_csv("data/fips_state.csv")

In [43]:
fips_state_dict = dict(list(zip(fips_state.fips, fips_state.abbr.str.upper())))

In [49]:
df['fips_code_state'] = df['fips_code']//1000

<ipython-input-49-85bc064860f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips_code_state'] = df['fips_code']//1000


In [53]:
df['property_state'] = df['property_state'].fillna(df['fips_code_state'].map(fips_state_dict))

<ipython-input-53-823d369fbab3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['property_state'] = df['property_state'].fillna(df['fips_code_state'].map(fips_state_dict))


In [54]:
df.property_state.isna().sum()

0

In [57]:
del df['fips_code_state']

A few records missing full_street_address when the information is all present.

In [55]:
df.loc[df.property_full_street_address.isna() & df.property_house_number.notna(), 'property_full_street_address'] = df['property_house_number'] + ' ' + df['property_street_name'] + ' ' + df['property_street_suffix']

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [56]:
#df[df.property_full_street_address.isna() & df.property_house_number.notna()]

A few records missing house_number in full_street_address when raw_house_number is present.

In [62]:
df.loc[df.dpid == "270890013503", 'property_full_street_address'] = "210 MAIN ST"

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [66]:
df.loc[df.dpid == "270890013505", 'property_full_street_address'] = "205 MAIN ST"

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [67]:
df['full_address'] = (df['property_full_street_address'] + ', '+
                      df['property_city_name'].fillna('') + ', ' +
                      df['property_state'].fillna('') + ', ' + 
                      df['property_zip_code'].fillna(''))

<ipython-input-67-8ab232681dca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_address'] = (df['property_full_street_address'] + ', '+


In [16]:
#df = df[['dpid', 'duplicateapn', 'full_address', 'property_full_street_address', 'property_city_name', 'property_state','property_zip_code', 'property_unit_type', 'property_unit_number','property_address_latitiude','property_address_longitude','property_address_census_tract']]

In [68]:
df['property_address_census_tract'] = df['property_address_census_tract'].astype(str)

<ipython-input-68-4452504abd96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['property_address_census_tract'] = df['property_address_census_tract'].astype(str)


In [69]:
df[[ 'full_address', 'property_full_street_address', 'property_city_name', 'property_state','property_zip_code', 'property_unit_type', 'property_unit_number']].isna().sum()/len(df)

full_address                    0.272869
property_full_street_address    0.272869
property_city_name              0.047150
property_state                  0.000000
property_zip_code               0.050606
property_unit_type              0.985983
property_unit_number            0.986033
dtype: float64

In [70]:
df.to_csv("data/BK_RAW_okay_records_no_vacant.csv", index = False)

In [3]:
df = pd.read_csv("data/BK_RAW_okay_records_no_vacant.csv", low_memory = False)

In [4]:
df[df.property_unit_type.notna()]

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
214,191930016735,19193,16780,884705145001,NaN,NaN,1529 S PALMETTO ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,PETERS INVESTMENT PROPERTIES LLC,NaN,NaN,NaN,3,"1529 S PALMETTO ST, SIOUX CITY, IA, 51106"
215,191930016736,19193,16781,884705145002,NaN,NaN,1529 S PALMETTO ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,PETERS INVESTMENT PROPERTIES LLC,NaN,NaN,NaN,3,"1529 S PALMETTO ST, SIOUX CITY, IA, 51106"
216,191930016737,19193,16782,884705145003,NaN,NaN,1529 S PALMETTO ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,PETERS INVESTMENT PROPERTIES LLC,NaN,NaN,NaN,3,"1529 S PALMETTO ST, SIOUX CITY, IA, 51106"
217,191930016738,19193,16783,884705145004,NaN,NaN,1529 S PALMETTO ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,PETERS INVESTMENT PROPERTIES LLC,NaN,NaN,NaN,3,"1529 S PALMETTO ST, SIOUX CITY, IA, 51106"
218,191930016739,19193,16784,884705146171,NaN,NaN,1517 S PALMETTO ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,SISSEL PROPERTIES LLC,NaN,NaN,NaN,3,"1517 S PALMETTO ST, SIOUX CITY, IA, 51106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249724,191930015728,19193,15761,884629276002,NaN,C,111 HILL ST,BRONSON,IA,51007.0,...,22.0,14,SCOTT,E,SCHROEDER,MICHELLE,NaN,SCHROEDER,3,"111 HILL ST, BRONSON, IA, 51007"
249763,191930015764,19193,15797,884629300003,NaN,C,1879 BRONSON BLACKTOP,BRONSON,IA,51007.0,...,8043.0,14,NaN,NaN,PANSEGRAU MARVIN L REVOCABLE TRUST,NaN,NaN,NaN,3,"1879 BRONSON BLACKTOP, BRONSON, IA, 51007"
249772,191930015769,19193,15802,884629300008,NaN,NaN,1963 BRONSON BLACKTOP,BRONSON,IA,51007.0,...,8043.0,14,CRAIG,T,GRIMSLEY,SONJA,R,GRIMSLEY,3,"1963 BRONSON BLACKTOP, BRONSON, IA, 51007"
249775,191930015835,19193,15867,884631200014,NaN,NaN,1809 BRONSON BLACKTOP,BRONSON,IA,51007.0,...,8041.0,14,L,JAY,KIRKPATRICK,LELSIE,NaN,KIRKPATRICK,3,"1809 BRONSON BLACKTOP, BRONSON, IA, 51007"


## Separate Bad Records (Records with bad addresses and empty addresses)

In [4]:
df_house_number = df[df.raw_property_house_number.notna() | df.property_house_number.notna()]
df_house_number

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
0,191930016537,19193,16584,884705119003,NaN,C,1810 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,LINDA,KAY,RIESENBERG,NaN,NaN,NaN,3,"1810 S LAKEPORT ST, SIOUX CITY, IA, 51106"
1,191930016538,19193,16585,884705119004,NaN,C,1814 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,NATHAN M,NaN,BROCKHAUS,NaN,NaN,NaN,1,"1814 S LAKEPORT ST, SIOUX CITY, IA, 51106"
2,191930016539,19193,16586,884705119005,NaN,C,1818 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,JARED,NaN,RAYMER,JOHN,NaN,RAYMER,1,"1818 S LAKEPORT ST, SIOUX CITY, IA, 51106"
3,191930016540,19193,16587,884705119006,NaN,NaN,1822 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,SOFIA,NaN,TARELO,ROBERTO,SOFIA,TARELO-GALLEGOS,3,"1822 S LAKEPORT ST, SIOUX CITY, IA, 51106"
4,191930016505,19193,16555,884705117021,NaN,C,1915 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,DAWN,A,LADE,NaN,NaN,NaN,3,"1915 S CLINTON ST, SIOUX CITY, IA, 51106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358488,191930016502,19193,16552,884705117018,NaN,C,1901 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,GARY,P,BOESHART,NaN,NaN,NaN,3,"1901 S CLINTON ST, SIOUX CITY, IA, 51106"
358489,191930016503,19193,16553,884705117019,NaN,NaN,1905 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,CHARLES,NaN,HACKER,MARY,NaN,HACKER,1,"1905 S CLINTON ST, SIOUX CITY, IA, 51106"
358490,191930016504,19193,16554,884705117020,NaN,C,1907 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,ADAM,NaN,VEGA,REBECCA,NaN,VEGA,1,"1907 S CLINTON ST, SIOUX CITY, IA, 51106"
358491,191930016536,19193,16583,884705119002,NaN,NaN,1806 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,PATRICIA,F,JACOBS,NaN,NaN,NaN,3,"1806 S LAKEPORT ST, SIOUX CITY, IA, 51106"


In [5]:
df_no_house_number = df[~df['dpid'].isin(df_house_number.dpid.unique())]

In [6]:
df_no_ad = df[df.property_full_street_address.isna()]
df_no_ad

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
177,191930063220,19193,60806,884705140005,NaN,NaN,NaN,SIOUX CITY,IA,51106.0,...,447.0,14,NaN,NaN,CITY OF SIOUX CITY,NaN,NaN,NaN,1,NaN
263,191670009207,19167,9235,08-33-376-001,NaN,NaN,NaN,HAWARDEN,IA,51023.0,...,NaN,12,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
270,191670033057,19167,31438,08-33-376-003,NaN,NaN,NaN,HAWARDEN,IA,51023.0,...,NaN,12,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
280,191670009213,19167,9241,08-34-126-002,NaN,NaN,NaN,NaN,IA,NaN,...,NaN,12,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
293,191670009250,19167,9278,08-35-426-001,NaN,NaN,NaN,NaN,IA,NaN,...,NaN,12,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358192,191930064091,19193,62396,884705118029,NaN,NaN,NaN,SIOUX CITY,IA,51106.0,...,447.0,14,NaN,NaN,CITY OF SIOUX CITY,NaN,NaN,NaN,3,NaN
358195,191930016157,19193,16192,884704176003,NaN,NaN,NaN,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
358208,191490014298,19149,47409,13-19-300-005,NaN,NaN,NaN,LE MARS,IA,51031.0,...,NaN,15,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
358322,191490028116,19149,47404,13-19-200-017,NaN,NaN,NaN,LE MARS,IA,51031.0,...,NaN,15,SHANE,MICHAEL,MARX,NaN,NaN,NaN,3,NaN


In [7]:
df_bad_records_key = set(df_no_ad.dpid.unique()).union(set(df_no_house_number.dpid.unique()))

In [8]:
df_bad_records = df[df['dpid'].isin(df_bad_records_key)]

In [9]:
df_bad_records

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
177,191930063220,19193,60806,884705140005,NaN,NaN,NaN,SIOUX CITY,IA,51106.0,...,447.0,14,NaN,NaN,CITY OF SIOUX CITY,NaN,NaN,NaN,1,NaN
263,191670009207,19167,9235,08-33-376-001,NaN,NaN,NaN,HAWARDEN,IA,51023.0,...,NaN,12,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
270,191670033057,19167,31438,08-33-376-003,NaN,NaN,NaN,HAWARDEN,IA,51023.0,...,NaN,12,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
280,191670009213,19167,9241,08-34-126-002,NaN,NaN,NaN,NaN,IA,NaN,...,NaN,12,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
282,191670009214,19167,9242,08-34-151-002,NaN,NaN,COOLIDGE AVE,HAWARDEN,IA,51023.0,...,NaN,12,NaN,NaN,"REHDER, JENNA L TRUST",NaN,NaN,"REHDER, KELLY",3,"COOLIDGE AVE, HAWARDEN, IA, 51023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358195,191930016157,19193,16192,884704176003,NaN,NaN,NaN,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
358207,191490028117,19149,47405,13-19-200-018,NaN,NaN,OTTER AVE,LE MARS,IA,51031.0,...,NaN,15,NaN,NaN,7 LLC,NaN,NaN,NaN,3,"OTTER AVE, LE MARS, IA, 51031"
358208,191490014298,19149,47409,13-19-300-005,NaN,NaN,NaN,LE MARS,IA,51031.0,...,NaN,15,NaN,NaN,NOT AVAILABLE FROM THE COUNTY,NaN,NaN,NaN,3,NaN
358322,191490028116,19149,47404,13-19-200-017,NaN,NaN,NaN,LE MARS,IA,51031.0,...,NaN,15,SHANE,MICHAEL,MARX,NaN,NaN,NaN,3,NaN


In [10]:
df_bad_records.to_csv("data/BK_RAW_bad_records.csv", index = False)

In [11]:
df = df[~df['dpid'].isin(df_bad_records_key)]

In [12]:
df

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
0,191930016537,19193,16584,884705119003,NaN,C,1810 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,LINDA,KAY,RIESENBERG,NaN,NaN,NaN,3,"1810 S LAKEPORT ST, SIOUX CITY, IA, 51106"
1,191930016538,19193,16585,884705119004,NaN,C,1814 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,NATHAN M,NaN,BROCKHAUS,NaN,NaN,NaN,1,"1814 S LAKEPORT ST, SIOUX CITY, IA, 51106"
2,191930016539,19193,16586,884705119005,NaN,C,1818 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,JARED,NaN,RAYMER,JOHN,NaN,RAYMER,1,"1818 S LAKEPORT ST, SIOUX CITY, IA, 51106"
3,191930016540,19193,16587,884705119006,NaN,NaN,1822 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,SOFIA,NaN,TARELO,ROBERTO,SOFIA,TARELO-GALLEGOS,3,"1822 S LAKEPORT ST, SIOUX CITY, IA, 51106"
4,191930016505,19193,16555,884705117021,NaN,C,1915 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,DAWN,A,LADE,NaN,NaN,NaN,3,"1915 S CLINTON ST, SIOUX CITY, IA, 51106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358488,191930016502,19193,16552,884705117018,NaN,C,1901 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,GARY,P,BOESHART,NaN,NaN,NaN,3,"1901 S CLINTON ST, SIOUX CITY, IA, 51106"
358489,191930016503,19193,16553,884705117019,NaN,NaN,1905 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,CHARLES,NaN,HACKER,MARY,NaN,HACKER,1,"1905 S CLINTON ST, SIOUX CITY, IA, 51106"
358490,191930016504,19193,16554,884705117020,NaN,C,1907 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,ADAM,NaN,VEGA,REBECCA,NaN,VEGA,1,"1907 S CLINTON ST, SIOUX CITY, IA, 51106"
358491,191930016536,19193,16583,884705119002,NaN,NaN,1806 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,PATRICIA,F,JACOBS,NaN,NaN,NaN,3,"1806 S LAKEPORT ST, SIOUX CITY, IA, 51106"


In [13]:
df.to_csv("data/BK_RAW_okay_records_no_vacant.csv", index = False)

## Find records with duplicate coordinates.
24070 rows

In [14]:
df_dup_coords = df[df.duplicated(['property_address_latitiude', 'property_address_longitude'], keep = False)]

In [15]:
df_dup_coords

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
9,191930016510 01,19193,62389,884705118002 01,1.0,NaN,1804 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,TERRY L,NaN,FOISTER,DIANE L,NaN,FOISTER,1,"1804 S CLINTON ST, SIOUX CITY, IA, 51106"
10,191930016510 02,19193,62390,884705118002 02,2.0,P,1804 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,M,FOISTER MICHAEL L FOISTER MELISSA,NaN,NaN,NaN,3,"1804 S CLINTON ST, SIOUX CITY, IA, 51106"
19,191930016521 01,19193,62391,884705118013 01,1.0,P,1811 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,KARI,NaN,TRIZILA,NaN,NaN,NaN,3,"1811 S LAKEPORT ST, SIOUX CITY, IA, 51106"
20,191930016521 02,19193,62392,884705118013 02,2.0,NaN,1811 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,VICTOR,NaN,LAMBERT,NaN,NaN,NaN,3,"1811 S LAKEPORT ST, SIOUX CITY, IA, 51106"
27,191930016530 01,19193,62393,884705118022 01,1.0,NaN,4617 MORNINGSIDE AVE,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,L,AHLERS JEFFREY J AHLERS LORI,NaN,NaN,NaN,3,"4617 MORNINGSIDE AVE, SIOUX CITY, IA, 51106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358362,191930016473 01,19193,62383,884705116008 01,1.0,P,1906 S NICOLLET ST,SIOUX CITY,IA,51106.0,...,NaN,14,CLARA,B,CALDERON,NaN,NaN,NaN,3,"1906 S NICOLLET ST, SIOUX CITY, IA, 51106"
358363,191930016473 02,19193,62384,884705116008 02,2.0,NaN,1906 S NICOLLET ST,SIOUX CITY,IA,51106.0,...,1192.0,14,NaN,NaN,HOFLING PROPERTIES LLC,NaN,NaN,NaN,3,"1906 S NICOLLET ST, SIOUX CITY, IA, 51106"
358365,191930016530 02,19193,62394,884705118022 02,2.0,NaN,4617 MORNINGSIDE AVE,SIOUX CITY,IA,51106.0,...,5394.0,14,NaN,M,MCMANAMY RONALD F MCMANAMY DIANNE,NaN,NaN,NaN,3,"4617 MORNINGSIDE AVE, SIOUX CITY, IA, 51106"
358480,191930016392 01,19193,62381,884705108009 01,1.0,C,4603 DAVIS AVE,SIOUX CITY,IA,51106.0,...,111.0,14,FERNANDO,NaN,RAYGOZA,NaN,NaN,NaN,3,"4603 DAVIS AVE, SIOUX CITY, IA, 51106"


## Records with duplicate coordinates and addresses (including NaN).
20483 cases

In [16]:
df_dup_coords_and_ad = df_dup_coords[df_dup_coords.duplicated(['property_address_latitiude', 'property_address_longitude', 'full_address'], keep = False)]

In [17]:
df_dup_coords_and_ad

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
9,191930016510 01,19193,62389,884705118002 01,1.0,NaN,1804 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,TERRY L,NaN,FOISTER,DIANE L,NaN,FOISTER,1,"1804 S CLINTON ST, SIOUX CITY, IA, 51106"
10,191930016510 02,19193,62390,884705118002 02,2.0,P,1804 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,M,FOISTER MICHAEL L FOISTER MELISSA,NaN,NaN,NaN,3,"1804 S CLINTON ST, SIOUX CITY, IA, 51106"
19,191930016521 01,19193,62391,884705118013 01,1.0,P,1811 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,KARI,NaN,TRIZILA,NaN,NaN,NaN,3,"1811 S LAKEPORT ST, SIOUX CITY, IA, 51106"
20,191930016521 02,19193,62392,884705118013 02,2.0,NaN,1811 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,VICTOR,NaN,LAMBERT,NaN,NaN,NaN,3,"1811 S LAKEPORT ST, SIOUX CITY, IA, 51106"
27,191930016530 01,19193,62393,884705118022 01,1.0,NaN,4617 MORNINGSIDE AVE,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,L,AHLERS JEFFREY J AHLERS LORI,NaN,NaN,NaN,3,"4617 MORNINGSIDE AVE, SIOUX CITY, IA, 51106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358362,191930016473 01,19193,62383,884705116008 01,1.0,P,1906 S NICOLLET ST,SIOUX CITY,IA,51106.0,...,NaN,14,CLARA,B,CALDERON,NaN,NaN,NaN,3,"1906 S NICOLLET ST, SIOUX CITY, IA, 51106"
358363,191930016473 02,19193,62384,884705116008 02,2.0,NaN,1906 S NICOLLET ST,SIOUX CITY,IA,51106.0,...,1192.0,14,NaN,NaN,HOFLING PROPERTIES LLC,NaN,NaN,NaN,3,"1906 S NICOLLET ST, SIOUX CITY, IA, 51106"
358365,191930016530 02,19193,62394,884705118022 02,2.0,NaN,4617 MORNINGSIDE AVE,SIOUX CITY,IA,51106.0,...,5394.0,14,NaN,M,MCMANAMY RONALD F MCMANAMY DIANNE,NaN,NaN,NaN,3,"4617 MORNINGSIDE AVE, SIOUX CITY, IA, 51106"
358480,191930016392 01,19193,62381,884705108009 01,1.0,C,4603 DAVIS AVE,SIOUX CITY,IA,51106.0,...,111.0,14,FERNANDO,NaN,RAYGOZA,NaN,NaN,NaN,3,"4603 DAVIS AVE, SIOUX CITY, IA, 51106"


## Records with duplicate coordinates and valid addresses. 
(So these records are legit. They should have the same coordinates anyway but somehow they are repeated. Some of them are co-owned so they are repeated.)
20483 cases

In [18]:
df_dup_coords_and_valid_ad = df_dup_coords_and_ad[df_dup_coords_and_ad['full_address'].notna()]

In [19]:
df_dup_coords_and_valid_ad

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
9,191930016510 01,19193,62389,884705118002 01,1.0,NaN,1804 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,TERRY L,NaN,FOISTER,DIANE L,NaN,FOISTER,1,"1804 S CLINTON ST, SIOUX CITY, IA, 51106"
10,191930016510 02,19193,62390,884705118002 02,2.0,P,1804 S CLINTON ST,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,M,FOISTER MICHAEL L FOISTER MELISSA,NaN,NaN,NaN,3,"1804 S CLINTON ST, SIOUX CITY, IA, 51106"
19,191930016521 01,19193,62391,884705118013 01,1.0,P,1811 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,KARI,NaN,TRIZILA,NaN,NaN,NaN,3,"1811 S LAKEPORT ST, SIOUX CITY, IA, 51106"
20,191930016521 02,19193,62392,884705118013 02,2.0,NaN,1811 S LAKEPORT ST,SIOUX CITY,IA,51106.0,...,NaN,14,VICTOR,NaN,LAMBERT,NaN,NaN,NaN,3,"1811 S LAKEPORT ST, SIOUX CITY, IA, 51106"
27,191930016530 01,19193,62393,884705118022 01,1.0,NaN,4617 MORNINGSIDE AVE,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,L,AHLERS JEFFREY J AHLERS LORI,NaN,NaN,NaN,3,"4617 MORNINGSIDE AVE, SIOUX CITY, IA, 51106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358362,191930016473 01,19193,62383,884705116008 01,1.0,P,1906 S NICOLLET ST,SIOUX CITY,IA,51106.0,...,NaN,14,CLARA,B,CALDERON,NaN,NaN,NaN,3,"1906 S NICOLLET ST, SIOUX CITY, IA, 51106"
358363,191930016473 02,19193,62384,884705116008 02,2.0,NaN,1906 S NICOLLET ST,SIOUX CITY,IA,51106.0,...,1192.0,14,NaN,NaN,HOFLING PROPERTIES LLC,NaN,NaN,NaN,3,"1906 S NICOLLET ST, SIOUX CITY, IA, 51106"
358365,191930016530 02,19193,62394,884705118022 02,2.0,NaN,4617 MORNINGSIDE AVE,SIOUX CITY,IA,51106.0,...,5394.0,14,NaN,M,MCMANAMY RONALD F MCMANAMY DIANNE,NaN,NaN,NaN,3,"4617 MORNINGSIDE AVE, SIOUX CITY, IA, 51106"
358480,191930016392 01,19193,62381,884705108009 01,1.0,C,4603 DAVIS AVE,SIOUX CITY,IA,51106.0,...,111.0,14,FERNANDO,NaN,RAYGOZA,NaN,NaN,NaN,3,"4603 DAVIS AVE, SIOUX CITY, IA, 51106"


## Records we want to focus on
1. Records with duplicated coordinates but different addresses
2. Records with duplicated addresses but different coordinates


### Subset 1: Records with duplicated coordinates but different addresses - 3608 cases
Generate a set of primary keys of the records with duplicate coordinates and valid street addresses.

In [20]:
df_dup_coords_and_valid_ad_pk_set = set(df_dup_coords_and_valid_ad.dpid.unique())

Exclude them from the records with duplicate coordinates. 

In [21]:
df_dup_coords_but_not_valid_ad = df_dup_coords[~df_dup_coords['dpid'].isin(df_dup_coords_and_valid_ad_pk_set)]

In [22]:
df_dup_coords_but_not_valid_ad

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
2690,191930019261,19193,19241,884706205006,NaN,NaN,1602 COLLEGE CT A,SIOUX CITY,IA,51106.0,...,9019.0,14,RALPH,G,PEDERSEN,LUELLA,M,PEDERSEN,3,"1602 COLLEGE CT A, SIOUX CITY, IA, 51106"
2691,191930019262,19193,19242,884706205007,NaN,NaN,1602 COLLEGE CT B,SIOUX CITY,IA,51106.0,...,NaN,14,JOHN,NaN,ZIMMERER,NaN,NaN,NaN,3,"1602 COLLEGE CT B, SIOUX CITY, IA, 51106"
2692,191930019263,19193,19243,884706205008,NaN,NaN,1602 COLLEGE CT C,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,SCHIMA-PEDERSEN ELISABETH REVOCABLE INTER-VIVO...,NaN,NaN,NaN,3,"1602 COLLEGE CT C, SIOUX CITY, IA, 51106"
3024,191930019264,19193,19244,884706205009,NaN,NaN,1602 COLLEGE CT D,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,PEDERSEN TODD G REVOCABLE LIVING TRUST,NaN,NaN,PEDERSEN TODD G,3,"1602 COLLEGE CT D, SIOUX CITY, IA, 51106"
3025,191930019265,19193,19245,884706205010,NaN,NaN,1602 COLLEGE CT E,SIOUX CITY,IA,51106.0,...,NaN,14,TODD,NaN,PEDERSEN,NaN,NaN,NaN,3,"1602 COLLEGE CT E, SIOUX CITY, IA, 51106"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358007,191930064065,19193,62342,884703330011,NaN,P,2515 INSIGNIA CIR,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,HOLCOMB FAMILY REVOCABLE TRUST,NaN,NaN,HOLCOMB JOANNE B,3,"2515 INSIGNIA CIR, SIOUX CITY, IA, 51106"
358008,191930064066,19193,62343,884703330012,NaN,P,2516 INSIGNIA CIR,SIOUX CITY,IA,51106.0,...,NaN,14,NaN,NaN,PAUL R GREGORY TRUST,NaN,NaN,NaN,3,"2516 INSIGNIA CIR, SIOUX CITY, IA, 51106"
358064,191930064086,19193,62363,884703352005,NaN,P,2710 ALBATROSS RDG,SIOUX CITY,IA,51106.0,...,NaN,14,JERRY,NaN,SANDBERG,SHAWN,HEATHER,SANDBERG,3,"2710 ALBATROSS RDG, SIOUX CITY, IA, 51106"
358065,191930064087,19193,62364,884703352006,NaN,NaN,2700 ALBATROSS RDG,SIOUX CITY,IA,51106.0,...,NaN,14,BRENT R,NaN,JOHNSEN,JAMI A,NaN,JOHNSEN,1,"2700 ALBATROSS RDG, SIOUX CITY, IA, 51106"


Some of the records with the same coordinates but different addresses are apartments.

In [23]:
df_dup_coords_but_not_valid_ad.property_unit_number.notna().sum()

15

In [24]:
df_dup_coords_but_not_valid_ad.to_csv("data/prob_subset_1_raw.csv", index = False)

### Subset 2: Records with the same address but different coordinates - 2347 cases

In [25]:
df_dup_ad = df[df.duplicated(['full_address'], keep = False)]

In [26]:
df_dup_ad = df_dup_ad[df_dup_ad['full_address'].notna()]

In [27]:
df_dup_ad_but_diff_coords = df_dup_ad[~df_dup_ad['dpid'].isin(df_dup_coords_and_valid_ad_pk_set)]

In [31]:
focus_2 = df_dup_ad_but_diff_coords

In [32]:
focus_2 = focus_2[focus_2['property_unit_number'].isna()]

In [33]:
focus_2.to_csv('data/prob_subset_2_raw.csv', index = False)

In [34]:
focus_2

,dpid,fips_code,pid,apn,duplicateapn,property_address_source_flag,property_full_street_address,property_city_name,property_state,property_zip_code,...,raw_mailing_zip4,edition,owner1_first_name,owner1_middle_name,owner1_last_name,owner2_firstname,owner2_middle_name,owner2_last_name,parsed_owner_source_code,full_address
11739,191930027092,19193,27199,884826200003,NaN,NaN,5411 PATTON ST,SIOUX CITY,IA,51111.0,...,168.0,14,NaN,NaN,OEHLERKING CHARLES H REVOCABLE TRUST OEHLERKIN...,NaN,NaN,OEHLERKING CHARLES H,3,"5411 PATTON ST, SIOUX CITY, IA, 51111"
11741,191930027094,19193,27201,884826200701,NaN,NaN,5411 PATTON ST,SIOUX CITY,IA,51111.0,...,168.0,14,NaN,NaN,OEHLERKING CHARLES H REVOCABLE TRUST OEHLERKIN...,NaN,NaN,OEHLERKING CHARLES H,3,"5411 PATTON ST, SIOUX CITY, IA, 51111"
12889,191930029370,19193,29471,894410100002,NaN,NaN,2674 160TH ST,MOVILLE,IA,51039.0,...,8028.0,14,STANLEY,NaN,FLEWELLING,GLORIA,NaN,FLEWELLING,3,"2674 160TH ST, MOVILLE, IA, 51039"
14539,191930031646,19193,31772,894532405011,NaN,NaN,110 E MAIN ST,LAWTON,IA,51030.0,...,NaN,14,BETH,NaN,BELLINGHAUSEN,NaN,NaN,NaN,3,"110 E MAIN ST, LAWTON, IA, 51030"
14540,191930031648,19193,31774,894532405013,NaN,NaN,110 E MAIN ST,LAWTON,IA,51030.0,...,NaN,14,BETH,NaN,BELLINGHAUSEN,NaN,NaN,NaN,3,"110 E MAIN ST, LAWTON, IA, 51030"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354527,191930009348,19193,9352,874614100001,NaN,NaN,2207 OLD HWY 141,BRONSON,IA,51007.0,...,8088.0,14,NORMAN,L,WILLIAMS,NaN,NaN,NaN,3,"2207 OLD HWY 141, BRONSON, IA, 51007"
354643,191930061794,19193,59363,874614100012,NaN,NaN,2207 OLD HWY 141,BRONSON,IA,51007.0,...,8088.0,14,CODY,L,WILLIAMS,NORMAN,L,WILLIAMS,3,"2207 OLD HWY 141, BRONSON, IA, 51007"
356353,191930013060,19193,13093,884332279001,NaN,NaN,604 N 2ND AVE,ANTHON,IA,51004.0,...,NaN,14,ERIC,J,O'CONNELL,JENNIFER,A,O'CONNELL,3,"604 N 2ND AVE, ANTHON, IA, 51004"
356354,191930013061,19193,13094,884332279002,NaN,NaN,604 N 2ND AVE,ANTHON,IA,51004.0,...,8004.0,14,JOHN,NaN,BOOK,NaN,NaN,NaN,3,"604 N 2ND AVE, ANTHON, IA, 51004"
